In [69]:
import numpy as np
import pandas as pd
import dateutil as dtutil
import pytz
import six

In [70]:
ratings = pd.read_csv('ratings.csv', encoding='latin-1', sep=',|:')
ratings.head()

/var/folders/zd/t8pjtw1d1qn5brdrsy47_xbh0000gn/T/ipykernel_99679/3026495763.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('ratings.csv', encoding='latin-1', sep=',|:')


,UserId,ItemId,Rating
0,4baf0ac888,8b05db84f2,3
1,f3364fd7e6,8b05db84f2,2
2,72a8982204,8b05db84f2,5
3,d0171f7a9d,e40cec3cbe,5
4,e6dad08499,e40cec3cbe,5


In [71]:
mean = ratings.groupby(['UserId'],as_index = False, sort = False).mean()

ratings = pd.merge(ratings, mean, on='UserId', how='left', sort=False)

ratings['normalized'] = ratings['Rating_x'] - ratings['Rating_y']

ratings.head()

,UserId,ItemId,Rating_x,Rating_y,normalized
0,4baf0ac888,8b05db84f2,3,3.75,-0.75
1,f3364fd7e6,8b05db84f2,2,1.50,0.50
2,72a8982204,8b05db84f2,5,5.00,0.00
3,d0171f7a9d,e40cec3cbe,5,5.00,0.00
4,e6dad08499,e40cec3cbe,5,4.00,1.00


In [72]:
#values = np.zeros((len(user_hash), len(item_hash)))
table = {}
values = {}

In [73]:

item_hash = {}
user_hash = {}

for (_, row) in ratings.iterrows():
    user_id = row['UserId']
    item_id = row['ItemId']
    
    if user_id not in user_hash: 
        user_hash[user_id] = len(user_hash)
    if item_id not in item_hash: 
        item_hash[item_id] = len(item_hash)  

In [74]:
for (_, row) in ratings.iterrows():
    if row['UserId'] not in values: 
        values[row['UserId']] = {}
    if row['UserId'] not in table: 
        table[row['UserId']] = {}

    values[row['UserId']][row['ItemId']] = row['Rating_x']
    table[row['UserId']][row['ItemId']] = row['normalized']

In [75]:
def pearson_correlation(user, table):
    similarity = []
        
    I = [x for x in table[user]]

    z =  np.sqrt(np.sum(np.square([table[user][i] for i in I]).astype(float)))
        
    for (uid, items_map) in table.items():
        n = np.sum([table[user][i] * (items_map[i] if i in items_map else 0.) for i in I])
        d = np.sqrt(np.sum(np.square(list(items_map.values())).astype(float)))
        
        if n != 0.:
            c = n / (z*d)
        else:
            c = 0.
        similarity.append((uid, c))
    
    similarity.sort(key=lambda x: (x[1], x[0]), reverse=True)
    
    return similarity

cosine_similarity = pearson_correlation('28523fc4c6', table)

cosine_similarity[:10]

[('28523fc4c6', 1.0000000000000002),
 ('5149bb3281', 0.5803704306990815),
 ('1937ad25fc', 0.5026155365907203),
 ('08750e4950', 0.20483662259967592),
 ('2d21554615', 0.20483662259967575),
 ('2499fe7714', 0.20114557032045355),
 ('f3364fd7e6', 0.17739371879672491),
 ('652447ebee', 0.16096579589605048),
 ('fd1807a8e6', 0.14782809899727054),
 ('fa816501c9', 0.14782809899727054)]

In [82]:
def predict(N, user, table, items):
    similarity = pearson_correlation(user, table)[:(N + 1)]
    predictions = []

    for item in items:
        n = 0
        d = 0

        for (user, s) in similarity[1:]:
            if item in table[user]:
                n += s * table[user][item]
                d += s

        rating = n/d if d != 0. else 0.

        predictions.append((item, round(rating)))
        
    predictions.sort(key=lambda x: (x[1], x[0]), reverse=True)

    return predictions

items = ratings['ItemId'].drop_duplicates()
predict(1024, '28523fc4c6', values, items)

[('ff0dbb4f09', 5),
 ('fed6565985', 5),
 ('fe2b0a7827', 5),
 ('fe1c9b4cff', 5),
 ('fce54213f7', 5),
 ('fcbc08af53', 5),
 ('fc6b39e539', 5),
 ('faae744545', 5),
 ('fa8de73f37', 5),
 ('fa1e724db5', 5),
 ('f8f6c71952', 5),
 ('f8ea6e3122', 5),
 ('f89423faa6', 5),
 ('f5d3a206c5', 5),
 ('f5cf1116a1', 5),
 ('f5b97f99c6', 5),
 ('f5876fb5ae', 5),
 ('f3fa6cb508', 5),
 ('f3f551bc8e', 5),
 ('f32b5a4ca0', 5),
 ('f2dab38cd8', 5),
 ('f2bf49945e', 5),
 ('f2499a4657', 5),
 ('f20d378d3e', 5),
 ('f1dfaf5930', 5),
 ('f10f001c97', 5),
 ('f0ffe18eea', 5),
 ('ef9b2d619a', 5),
 ('ef7972253f', 5),
 ('ef59fcf441', 5),
 ('ef52365ce0', 5),
 ('eefb13d375', 5),
 ('ee70a75353', 5),
 ('ed9b2bc206', 5),
 ('ed7489d4c9', 5),
 ('ed4cac4da3', 5),
 ('ec1980e3d0', 5),
 ('eab7bbf190', 5),
 ('ea8966d843', 5),
 ('e9ca7ca609', 5),
 ('e901903ec0', 5),
 ('e8ccb84d5f', 5),
 ('e877b901ba', 5),
 ('e84cd1275f', 5),
 ('e725d0a208', 5),
 ('e6b76eb51c', 5),
 ('e5352e1117', 5),
 ('e4e1d83893', 5),
 ('e448f0a6d9', 5),
 ('e2a8f1fe4c', 5),
